In [1]:
import pandas as pd
import glob

In [12]:
from configs import config_generator

config_set_list = [
    config_generator.generate_resnet_mlb_configs(True),
    config_generator.generate_retinanet_mlb_configs(True),
    config_generator.generate_maskrcnn_mlb_configs(True),
    config_generator.generate_bert_mlb_configs(True)
]

In [13]:
import re

def block_repeat(filepath):
    match = re.search(r'(\d+)(?=\.\w+$)', filepath)
    if match:
        return int(match.group(1))
    return None

def extract_l_hs(filename):
    pattern = r'l_(\d+)_hs_(\d+)\.csv'
    match = re.search(pattern, filename)
    if match:
        l = int(match.group(1))
        hs = int(match.group(2))
        return l, hs
    else:
        return None

In [15]:
df_all_list = []
for config_set in config_set_list:

    exp_dir_path = f'./intensity-experiments/{config_set[0].experiment_dir}/'
    result_paths = glob.glob(exp_dir_path+'*')
    model_arch = config_set[0].model_arch
    is_train = 'train' if config_set[0].is_train else 'inference'

    df_list = []
    for path in result_paths:
        curr_df = pd.read_csv(path)
        if model_arch == 'bert':
            extracted = extract_l_hs(path)
            if extracted:
                l, hs = extracted
                curr_df['num_layers'] = l
                curr_df['hidden_size'] = hs
        else:
            extracted = block_repeat(path)
            if extracted:
                curr_df['block_repeat'] = extracted
        df_list.append(curr_df)

    if model_arch == 'bert':
        results_df = pd.concat(df_list, ignore_index=True).sort_values(by=['hidden_size','num_layers'])
        results_df.to_csv(f'{exp_dir_path}l_hs_aggregated_{is_train}.csv', index=False)
    else:
        results_df = pd.concat(df_list, ignore_index=True).sort_values(by='block_repeat')
        results_df.to_csv(f'{exp_dir_path}block_repeat_aggregated_{is_train}.csv', index=False)
    
    df_all_list.append(results_df)

df_all = pd.concat(df_all_list, ignore_index=True).sort_values(by=['block_repeat', 'hidden_size', 'num_layers'])
df_all.to_csv(f'./intensity-experiments/{is_train}_aggregated_ver1.0.csv', index=False)


In [9]:
results_df

,flops,imem,pmem,ai_estimate,num_layers,hidden_size
12,7.621672e+09,3.944859e+08,1.783492e+08,13.305176,12,360
0,7.974090e+09,4.168625e+08,1.892954e+08,13.155137,13,360
2,1.052655e+10,4.775970e+08,2.492260e+08,14.482969,12,480
14,1.102540e+10,5.054307e+08,2.647384e+08,14.315556,13,480
11,1.361760e+10,5.664680e+08,3.258628e+08,15.260708,12,600
3,1.427762e+10,6.002197e+08,3.464023e+08,15.082703,13,600
1,1.689481e+10,6.610990e+08,4.082596e+08,15.799011,12,720
13,1.773074e+10,7.012294e+08,4.342869e+08,15.614698,13,720
4,2.138478e+10,8.084600e+08,5.283923e+08,15.996366,13,840
21,2.035818e+10,7.614901e+08,4.964164e+08,16.184177,12,840
